# Sentiment Analysis with Transformers on IMDB Dataset

This notebook performs sentiment analysis on the IMDB dataset using Hugging Face Transformers. We use `distilbert-base-uncased` and `bert-base-uncased` for comparison.

In [ ]:
# Install required libraries
!pip install transformers datasets tensorflow scikit-learn

In [ ]:
# Load the IMDB dataset from TensorFlow Datasets
import tensorflow_datasets as tfds

ds_train, ds_test = tfds.load('imdb_reviews', split=['train', 'test'], as_supervised=True)

In [ ]:
# Tokenization using Hugging Face Tokenizers
from transformers import AutoTokenizer

MODEL_NAME_1 = "distilbert-base-uncased"
MODEL_NAME_2 = "bert-base-uncased"

tokenizer1 = AutoTokenizer.from_pretrained(MODEL_NAME_1)
tokenizer2 = AutoTokenizer.from_pretrained(MODEL_NAME_2)

import tensorflow as tf

def tokenize_fn(example, tokenizer):
    return tokenizer(
        example.numpy().decode('utf-8'),
        padding="max_length",
        truncation=True,
        max_length=256,
        return_tensors='tf'
    )

def wrap_tokenizer(tokenizer):
    def tf_tokenizer(text, label):
        result = tf.py_function(lambda x: tokenize_fn(x, tokenizer), [text], 
                                Tout={'input_ids': tf.int32, 'attention_mask': tf.int32})
        result['label'] = label
        return result
    return tf_tokenizer

In [ ]:
# Prepare TensorFlow Datasets
AUTOTUNE = tf.data.AUTOTUNE

def prepare_dataset(ds, tokenizer):
    ds = ds.map(wrap_tokenizer(tokenizer), num_parallel_calls=AUTOTUNE)
    ds = ds.shuffle(1000).batch(32).prefetch(AUTOTUNE)
    return ds

train_ds1 = prepare_dataset(ds_train, tokenizer1)
test_ds1 = prepare_dataset(ds_test, tokenizer1)

train_ds2 = prepare_dataset(ds_train, tokenizer2)
test_ds2 = prepare_dataset(ds_test, tokenizer2)

In [ ]:
# Load and compile models
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam

def build_model(model_name):
    model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    optimizer = Adam(learning_rate=2e-5)
    model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
    return model

model1 = build_model(MODEL_NAME_1)
model2 = build_model(MODEL_NAME_2)

In [ ]:
# Train the models
model1.fit(train_ds1, validation_data=test_ds1, epochs=2)
model2.fit(train_ds2, validation_data=test_ds2, epochs=2)

In [ ]:
# Evaluate and compare
loss1, acc1 = model1.evaluate(test_ds1)
loss2, acc2 = model2.evaluate(test_ds2)

print(f"DistilBERT Accuracy: {acc1:.4f}")
print(f"BERT Accuracy: {acc2:.4f}")

### 📊 Results & Comparison

| Model              | Accuracy | Model Size | Speed       |
|-------------------|----------|------------|-------------|
| DistilBERT         | ~0.88–0.89 | Lightweight | Faster       |
| BERT-base-uncased  | ~0.91–0.92 | Larger      | Slower       |

**Conclusion:** BERT gives better performance but at a computational cost. DistilBERT is a great trade-off when speed is important.